In [ ]:
import sys;
from sqes_function import Config
import psycopg2
import pandas as pd
from obspy import read_inventory

import warnings
warnings.filterwarnings("ignore", message="pandas only supports SQLAlchemy connectable")

In [ ]:
db_config = Config.load_config(section="postgresql") 
engine = psycopg2.connect(**db_config)

In [ ]:
stations_db = pd.read_sql('select code,latitude,longitude from stations', con=engine)

In [ ]:
# collect data from xml
rows_list = []
for code in stations_db.code:
    inv = read_inventory(f"https://geof.bmkg.go.id/fdsnws/station/1/query?station={code}&level=response&nodata=404")
    dict_xml = {
    'code': inv[0].stations[0].code,
    'latitude_xml': inv[0].stations[0].latitude,
    'longitude_xml' : inv[0].stations[0].longitude
    }
    rows_list.append(dict_xml)
stations_xml = pd.DataFrame(rows_list)

# merge
merge = stations_db.join(stations_xml.set_index('code'), on='code')
merge['latitude_diff'] = abs(merge.latitude - merge.latitude_xml)
merge['longitude_diff'] = abs(merge.latitude - merge.latitude_xml)

In [ ]:
# check station lat and lon difference based on threshold
threshold = 0 #1/111
merge[(merge.latitude_diff > threshold) | (merge.longitude_diff > threshold)]

In [ ]:
# update db
sql="""UPDATE stations SET latitude=%s, longitude=%s WHERE code=%s"""
rowcount=0
for code in stations_db.code:
    print("processing: ",code)
    # take data from station xml
    inv = read_inventory(f"https://geof.bmkg.go.id/fdsnws/station/1/query?station={code}&level=response&nodata=404")
    dict_xml = {
    'code': inv[0].stations[0].code,
    'latitude': inv[0].stations[0].latitude,
    'longitude' : inv[0].stations[0].longitude
    }
    # processing diff
    db_lat = stations_db[stations_db.code==code].latitude.values[0]
    db_lon = stations_db[stations_db.code==code].longitude.values[0]
    diff_lat=db_lat-dict_xml['latitude']
    diff_lon=db_lon-dict_xml['longitude']
    print(f"latitude_def : {db_lat: >10} | latitude_xml : {dict_xml['latitude']: >10} | diff: {diff_lat}")
    print(f"longitude_def: {db_lon: >10} | longitude_xml: {dict_xml['longitude']: >10} | diff: {diff_lon}")
    
    # update db
    with engine.cursor() as cur:
        cur.execute(sql,(dict_xml['latitude'],dict_xml['longitude'],code))
        rowcount=cur.rowcount
    engine.commit()
    print(f"updated db row {rowcount}")
    
    print(f"end of {code}")
    print("--------------------------------------------------------------------------------------------")
print("process finish")